In [1]:
# Importing libraries
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
import pandas.io.sql as sqlio
import psycopg2
import json
import sys
sys.path.extend([
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava',
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava\Scripts'
])
from Functions import *
from StravaCredentials import *

# Creating connection to database and creating engine to upload data
db_url = f'postgresql://{username}:{pwd}@{hostname}:{port_id}/{database}'
engine = create_engine(db_url)

conn = psycopg2.connect(
    host = hostname,
    dbname = database,
    user = username,
    password = pwd,
    port = port_id
)

# Creating date variable
#today = datetime.today().strftime('%Y-%m-%d')

# Getting Strava acccess token
access_token = GetToken(data)

# Getting an updated list of activities
# activities df - this does not include workout details
updated_workouts = retrieve_activities(access_token)

# Creating list to compare against not updated list
updated_workouts_list = list(updated_workouts['id'])


# Creating a cursor and also quering the database to get the current list of workouts from activity table
cur = conn.cursor()
query = "select activity_id from activity"

# Turning query into dataframe - all activities
not_updated_workouts = sqlio.read_sql_query(query, conn)
print('Extracting data from SQL database.')

# Creating a list of ids from the list of activity ids
not_updated_workouts_list = list(not_updated_workouts['activity_id'])

# How many new workouts will be added to database
print('Adding',len(updated_workouts_list) - len(not_updated_workouts_list),'workouts to Database.')

# Creating a list containing the missing workouts
# missing workouts = workouts to be added 
missing_workouts = list(set(updated_workouts_list).difference(not_updated_workouts_list))

Getting page number: 1
Extraction of page 1 complete
Getting page number: 2
Extraction of page 2 complete
Getting page number: 3
Extraction of page 3 complete
Getting page number: 4
Extraction of page 4 complete
Getting page number: 5
Extraction of page 5 complete
Getting page number: 6
Extraction of page 6 complete
Extraction done
Extracting data from SQL database.
Adding 9 workouts to Database.


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14428\1298297996.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  not_updated_workouts = sqlio.read_sql_query(query, conn)


In [4]:
# Calling function to extract missing workouts
missing_workouts_json = GetAllWorkouts(missing_workouts, access_token)

Extracting all workouts, due to the API rate limit, this will take 1.35 minutes.
Extracting workout: 1
Extracting workout: 2
Extracting workout: 3
Extracting workout: 4
Extracting workout: 5
Extracting workout: 6
Extracting workout: 7
Extracting workout: 8
Extracting workout: 9


In [5]:
# Parsing workout json
missing_workouts_df = CleanWorkoutJson(missing_workouts_json)

In [7]:
# Parsing workout json to create laps table
laps_df = CleanLapsJson(missing_workouts_json)

# Creating the score columns from the workout json 
missing_workouts_df = CreateScoreColumns(missing_workouts_df)

In [10]:
# Dividing all_workouts_df to multiple tables to then upload to database
activity, activity_name, activity_coordinates, activity_details, activity_scores = DivideTables(missing_workouts_df)

In [14]:

# Sending data to Postgresql
print('Uploading data to database.')

activity.to_sql('activity', engine, if_exists='append', index=False)
activity_name.to_sql('activity_name', engine, if_exists='append', index=False)
activity_coordinates.to_sql('activity_coordinates', engine, if_exists='append', index=False)
activity_details.to_sql('activity_details', engine, if_exists='append', index=False)
activity_scores.to_sql('activity_scores', engine, if_exists='append', index=False)
laps_df.to_sql('laps', engine, if_exists='append', index=False)

print('Database updated.')

Uploading data to database.
Database updated.
